# ZFNet(Theory)

ZFNet是2013年ILSVRC([ImageNet Large Scale Visual Recognition Competition](http://www.image-net.org/challenges/LSVRC/))的winner,由纽约大学 Matthew D. Zeiler, Rob Fergus提出,实际上该年的冠军并不是这个模型,严格意义上来说当时分类冠军是Clarifai,但是我们通常讨论的ILSVRC2013冠军(winner)指的是ZFNet.ZF中的Zeiler是Clarifai的创建者和CEO.

ZFNet最大的作用是直观理解CNN为何有效,实际上我们知道AlexNet实力出众,但是当时人们并不知道CNN为什么会有如此好的效果,然而ZFNet给出了直观的理由.

另外ZFNet和AlexNet很相似,只是做以一点点的微调.

Paper:

[Visualizing and Understanding Convolutional Networks](https://arxiv.org/pdf/1311.2901.pdf)

### 1. Visualization with a Deconvnet

ZFNet给出了一种可以直观解释CNN的方法, Visualization with a Deconvnet,但是需要注意的是,虽然方法叫做Deconvent,但是其不是CNN反向传播(梯度更新)的意思,准确来说,现在我们将其叫做Transposed Convolution以避免混淆.

下面我们来说说Transposed Convolution的具体流程:

<img src="../../../picture/214.png" height=500 width=500>

图中右边是一个正常的卷积过程:**CONV==>Activation==> MaxPooling**,使用Switches记录右边MaxPooling中最大值的位置,然后左边则是Transposed Convolution:

[1] Max Uppooling:

上池化(反池化):我们知道Max Pooling是不可逆的,在此之前对于Pooling层采用的是[此方法](http://www.xml-data.org/GDGYDXXB/html/20170604.htm#outline_anchor_16),而论文作者给出了一个近似的方式(也就是我们CNN Forward and Backward(Theory01)使用的方式)

<img src="../../../picture/215.png" height=500 width=500>


[2] Rectiled Unpooled Maps 

得到了Max Pooling之后,为了重构出有效的特征,也可以说是因为在正常卷积过程中使用了ReLu,所以这里也会使用ReLu,从而得到Rectiled Unpooled Maps.

[3] Reconstruction

最后使用原始卷积核的**转置**$Convolution\;Filtering \{F^{T}\}$进行卷积操作得到重构图,正是由于这一步使用的是转置的原卷积核所以我们将其称之为Transpose Convolution,在Tensorflow[tf.nn.conv2d_transpose](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d_transpose)中也有提及:

<img src="../../../picture/216.png" height=500 width=500>

更多关于[deConvolution](https://www.matthewzeiler.com/mattzeiler/deconvolutionalnetworks.pdf)

注意为了获得重构图,我们的Transpose Convolution直观理解:

<h4>Conv (Blue is input, cyan is output)</h4>
<img src="https://cdn-images-1.medium.com/max/1600/0*uxsQQN6UtlxksaDX" height=300 width=300>
<h4>Transpose Convolution (Blue is input, cyan is output)</h4>
<img src="https://cdn-images-1.medium.com/max/1600/0*CJYLcAXhmOepbMmh" height=300 width=300>

为检验一个给定CNN的激活,我们就将该激活的feature map后接一个反卷积网络,然后通过:反池化、反激活、反卷积.重复上述过程直到原始输入层:

假如你想要查看Alexnet 的conv5提取到了什么东西,我们就用conv5的特征图后面接一个反卷积网络,然后通过:反池化、反激活、反卷积,这样的一个过程,把本来一张13\*13大小的特征图(conv5大小为13\*13),放大回去,最后得到一张与原始输入图片一样大小的图片(227*227).


这样就能够重建Conv的特征图(features maps),接下去我们来看看作者使用Reconstruction对AlexNet的分析.

### 2. 特征重现

<img src="../../../picture/217.png" height=800 width=800>


1.上图是对ImageNet训练完成后,在验证集数据进行反卷积得到的各层的可视化结果.

2.显示了随机验证集中前9个,让**特定的**(多个featuremap中的一个其他全重置为0)feature map响应最大的可视化结果,画成一个九宫格.把它们映射回输入空间后可以看到不同结构的重建特征图(灰色的那些图),以及这些特征图对应图像块(那些彩色图).

3.可以看到彩色图的变化比灰色图更大,因为灰色图是集中于给出那些具有区分性的信息.

4.可以看到每一层似乎在学习不同的东西,第二层学习边缘,角落信息;第三层学到了一些比较复杂的模式,网状,轮胎;第四层展示了一些比较明显的变化,但是与类别更加相关了,比如狗脸,鸟腿;第五层则看到了整个物体,比如键盘,狗.也就是说,第二层应对角落和其他边缘或者颜色的结合;第三层有更加复杂的不变性,捕捉到了相似的纹理;第四层显示了特定类间显著的差异性;第五层显示了有显著构成变化的整个物体.

5.显示了9个让特定featuremap响应最大的可视化结果.从而展示其中的不变性.一些固定的输入特征刺激神经网络产生固定的输出特征(隐含的),这也解释了图像畸变时候,为什么输出结果保持不变.和重构特征相比,输入图片之间的差异很大,而重构特征只包含哪些具有判别能力的纹理结构.举例说明,第5层第1行第二列,9张图片个不相同,重构出来的特征显示的是背景的草地,而不是前景特征.


### 3. 训练过程的特征演化

![](../../../picture/218.png)

1.上图是随着训练的迭代,特征图的变化,每一层里面的8列表示不同epoch时的特征图.

2.外表突然的变化导致图像中的一个变换即产生了最强烈的激活.模型的底层在少数几个epoches就能收敛聚集,然而上层在一个相当多的epoches(40-50)之后才能有所变化,这显示了让模型完全训练到完全收敛的必要性.可以由上图看到颜色对比度都逐步增强.

### 4. 特征不变性

![](../../../picture/219.png)

1.一般来说,小的变化对于模型的第一层都有非常大的影响,但对于最高层的影响却几乎没有.对于图像的平移、尺度、旋转的变化来说,网络的输出对于平移和尺度变化都是稳定的,但却不具有旋转不变性,除非目标图像时旋转对称的.下图为分别对平移,尺度,旋转做的分析图.

2.上图按行顺序分别为对5类图像进行不同程度的垂直方向上的平移、尺度变换、旋转对输出结果影响的分析图.按列顺序分别为原始变换图像,第二列为第一层中原始图片和变换后的图片的欧氏距离,可以看到距离都非常大,对于尺度(scale)最大能够达到10以上.

3.第三列为第7层中原始图片和变换后的图片的欧氏距离,可以看到欧式距离都相差不是很大.

4.第四列为变换后图片被正确分类的概率图,可以看到对于平移,尺度在一定范围内正确率受影响不是很验证,但是旋转而言受影响程度还是较大的.

### 5. ZFNet

那么实际上ZFNet只是对AlexNet进行微调:

![](../../../picture/220.png)

**微调内容:**

和AlexNet一样,头两个全连接层后面加0.5的dropout.相比于AlexNet,主要区别是:

(1) Conv1:采用7x7的卷积核，步长为2(AlexNet采用11x11的卷积核，步长为4).

(2) 将网络整体变宽:Conv3,4,5的feature map个数变成:512,1024,512,而不是384,384,256.

通过可视化发现第一层的卷积核影响大,于是对第一层的卷积核做了规范化,比如RMS(Root Mean Square)超过0.1,就把卷积核的均方根normalize为固定0.1.

2.权重初始化不同,权重初始化为0.01,bias初始化为0.而AlexNet为:SGD,学习率0.01,batch size是128,momentum为0.9,weight decay为0.0005,每当validation error不再下降时,学习率除以10.权重初始化用(0,0.01)的高斯分布,二四五卷积层和全连接层的bias初始化为1(给relu提供正值利于加速前期训练),其余bias初始化为0.

具体参数如下:

<table border="0">
	<tbody>
        <tr>
			<td style="text-align: center"><strong>网络层</strong></td>
			<td style="text-align: center"><strong>卷积核尺寸</strong></td>
            <td style="text-align: center"><strong>步长</strong></td>
            <td style="text-align: center"><strong>填充</strong></td>
            <td style="text-align: center"><strong>输出维度</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Input(224)</strong></td>
			<td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
        </tr>
         <tr>
			<td style="text-align: center"><strong>Conv1</strong></td>
			<td style="text-align: center"><strong>7x7</strong></td>
            <td style="text-align: center"><strong>2</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>96</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Pool1</strong></td>
			<td style="text-align: center"><strong>3x3</strong></td>
            <td style="text-align: center"><strong>2</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>96</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Conv2</strong></td>
			<td style="text-align: center"><strong>5x5</strong></td>
            <td style="text-align: center"><strong>2</strong></td>
            <td style="text-align: center"><strong>0</strong></td>
            <td style="text-align: center"><strong>256</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Pool2</strong></td>
			<td style="text-align: center"><strong>3x3</strong></td>
            <td style="text-align: center"><strong>2</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>256</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Conv3</strong></td>
			<td style="text-align: center"><strong>3x3</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>384</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Conv4</strong></td>
			<td style="text-align: center"><strong>3x3</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>384</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Conv5</strong></td>
			<td style="text-align: center"><strong>3x3</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>1</strong></td>
            <td style="text-align: center"><strong>256</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Pool5</strong></td>
			<td style="text-align: center"><strong>3x3</strong></td>
            <td style="text-align: center"><strong>2</strong></td>
            <td style="text-align: center"><strong>0</strong></td>
            <td style="text-align: center"><strong>256</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Fc6</strong></td>
			<td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>4096</strong></td>
        </tr>
        <tr>
			<td style="text-align: center"><strong>Fc7</strong></td>
			<td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>-</strong></td>
            <td style="text-align: center"><strong>4096</strong></td>
        </tr>
	</tbody>
</table>

需要注意的是在Tensorflow中,我们输入是(225x225),因为如果是224的输入那么按照卷积核的大小和步长以及空洞为1,是无法输出(110x110)的.

在ZFNet训练完毕后对于验证集可视化:

![](../../../picture/222.png)

上图(a)为没有经过裁剪的图片经过第一个卷积层后的特征可视化图,注意到有一个特征全白.(b)为AlexNet中第一个卷积层特征可视化图,(c)为ZFNet中第一个卷积层可视化图,可以看到相比前面有更多的独特的特征以及更少的无意义的特征,如第3列的第3到6行.(d)为AlexNet中第二个卷积层特征可视化图,(e)为ZFNet中的第二个卷积层特征可视化图,可以看到(e)中的特征更加干净,清晰,保留了更多的第一层和第二层中的信息.

接着我们可以询问:**整个模型是否知道目标在图像中的具体位置?**针对于这一点,论文做了以下实验:

![](../../../picture/223.png)

上图为对三个不同的测试图片中的不同位置用灰色小方块进行掩盖后,观测分类输出的改变情况.第一列(a)为原始测试图片,第二列(b)为某个区域被灰色小方块掩盖后的网络结构中**第五层**的特征图,第三列(c)为将第五层的特征图投影到输入图像的可视化图像,(d)为正确分类概率的图,(e)为最有可能的标签.

以第一张小狗图为例,b图表示第5层激活最强的feature map,每个位置的颜色表示那个位置被遮挡后的激活,可以看到遮住狗脸后,激活值最低,也就是蓝色那块区域,说明遮挡对模型有影响,c图表示被遮住不同区域后第5层激活最强的feature map,第一个图表示遮住狗脸后的可视化,d图中每个位置的颜色表示那个位置被遮挡后,正确类别的概率.可以看到遮住狗脸后,概率很低,蓝色那一块,说明遮挡对模型有影响.e图中每个位置的颜色表示那个位置被遮挡后,最可能的标签,可以看到如果遮挡其它区域,模型都能识别出为博美犬,但是如果遮住了狗脸,而不遮住球,模型就会将样本分类为球.

上述结果表明,如果图像中的目标被遮挡,那么被正确分类的概率会显著降低,这展示了模型的遮挡敏感性,模型确实学到了物体的位置,而不是说只学到物体的环境上下文.


### 6. 进一步探究

接着为了进一步探究掩盖对同一类物体的不同图像进行一致性分析,论文使用了[Hamming distance](https://zh.wikipedia.org/wiki/%E6%B1%89%E6%98%8E%E8%B7%9D%E7%A6%BB)简单的说,汉明距离就是两个字符串对应位置的不同字符的个数,比如在维基百科上的例子:

<img src="../../../picture/224.png" height=300 width=300>

对五张小狗(同一类)的不同图片的不同区域进行掩盖,然后进行分析观察探究深度模型是对一类物体的那部分进行一致性分析的过程:

<img src="../../../picture/225.png" height=500 width=500>

由上图可以观察到,在第5层随机遮挡的情况比其他眼睛鼻子被遮挡的情况一致性分析较差(因为各个掩盖部分的汉明距离差距过大),而第7层中,这四类的汉明距离却都差不多,那是因为底层判别的是一类物体共有的部分(卷积核参数共享),而高层判别的是类别中不同的品种这种更高维的部分了.

最后论文对对不同的数据集进行了验证这里就不在赘述了,详情查看论文.

### 7. Summary

1.提出了一种可视化CNN的方法,说明了内部特征并不是随机的,是可以解释的.

2.通过可视化CNN了解到了一些直觉上的特性,比如随着增加层数,类别的可区分度越高,特征越有用.

3.通过可视化CNN可以对模型进行分析和改善.

4.通过可视化CNN的遮蔽实验,发现模型对局部结构是敏感的,并不是只用到了广阔的场景信息.

5.展示了ImageNet的预训练模型可以很好地泛化到其它数据集,也侧面说明了ImageNet数据集的牛逼性.

6.实际上CNN的可视化方法非常多,deConv只是最简单的一种,还有许多其他的方法得到的结果会比deConv更加清晰比如以下的几个链接:

[Visualization of Deep Covolutional Neural Networks](https://github.com/conan7882/CNN-Visualization)

[Convolutional Neural Network Visualizations](https://github.com/utkuozbulak/pytorch-cnn-visualizations)

[利用可视化方法直观理解CNN](https://zhuanlan.zhihu.com/p/39822145)

实际上可视化并不只是可以用来观测CNN的过程,还可以做一些"艺术",在后面我们将慢慢道来.



**Reference:**

[1] [Visualizing and Understanding Convolutional Networks](https://arxiv.org/pdf/1311.2901.pdf)

[2] [论文笔记:Visualizing and Understanding Convolutional Networks](https://www.cnblogs.com/liaohuiqiang/p/9356753.html)

[3] [Review: ZFNet — Winner of ILSVRC 2013 (Image Classification)](https://medium.com/coinmonks/paper-review-of-zfnet-the-winner-of-ilsvlc-2013-image-classification-d1a5a0c45103)

[4] [深度学习之可视化ZFNet-解卷积](https://blog.csdn.net/qq_31531635/article/details/71104334)

[5] [如何理解深度学习中的deconvolution networks？](https://www.zhihu.com/question/43609045)